In [1]:
import puzzlerepresentation as P

In [2]:
import numpy as np

In [3]:
history = {}

In [4]:
def addChromosomeToHistory(C):
    history[C.tostring()] = 5000

In [5]:
h = (6,0)
null_C = np.zeros([10,3],dtype=np.int8)
if h:
    null_C[-1] = [h[0], h[1], 0]
addChromosomeToHistory(null_C)

In [6]:
def buildRandomChromosome(hole=None):
    C = np.random.randint(0, 7, size=(10,3), dtype=np.int8)
    C[:,-1] = np.random.randint(0,4, size=10, dtype=np.int8)
    if hole:
        C[-1] = [hole[0], hole[1], 0]
    if C.tostring() in history:
        return buildRandomChromosome(hole=hole)
    addChromosomeToHistory(C)
    return C

In [7]:
h = (6,0)

In [8]:
pieces = P.pieces + [P.Piece('black',[[0,0]])]

In [55]:
def score(C):
    #if history[C.tostring()] != 5000:
    #    return history[C.tostring()]
    A = np.zeros([7,7],dtype=np.int8)
    #pieces = P.pieces + [P.Piece('black',[[0,0]])]
    for i in range(10):
        for unit in pieces[i].rot_list(C[i,2]):
            x = C[i][0] + unit[0]
            y = C[i][1] + unit[1]
            if (min(x,y) < 0) or (max(x,y) > 6):
                return np.infty
            A[x][y] += 1
    #print(np.abs(A-1).sum())
    print(A)
    s = np.abs(np.square(A-1)).sum()
    history[C.tostring()] = s
    return s

In [10]:
def test_pieces(C):
    A = np.zeros([7,7],dtype=np.int8)
    for i in range(10):
        print(f'{i}: {pieces[i].color}, {C[i]}')
        print(pieces[i].rot_list(C[i,2]))
        for unit in pieces[i].rot_list(C[i,2]):
            x = C[i][0] + unit[0]
            y = C[i][1] + unit[1]
            if (min(x,y) < 0) or (max(x,y) > 6):
                print(f'pos=({x},{y}) for piece {i}')
                print(A)
                return np.infty
            A[x][y] += 1
        print(A)
        A = np.zeros([7,7],dtype=np.int8)

In [58]:
ans = np.zeros([10,3],dtype=np.int8)
ans[0]=[1,0,0]
ans[1]=[1,3,0]
ans[2]=[3,2,2]
ans[3]=[6,6,2]
ans[4]=[5,5,2]
ans[5]=[2,4,3]
ans[6]=[0,6,3]
ans[7]=[3,6,2]
ans[8]=[0,0,0]
ans[9]=[6,0,0]
addChromosomeToHistory(ans)

In [59]:
ans

array([[1, 0, 0],
       [1, 3, 0],
       [3, 2, 2],
       [6, 6, 2],
       [5, 5, 2],
       [2, 4, 3],
       [0, 6, 3],
       [3, 6, 2],
       [0, 0, 0],
       [6, 0, 0]], dtype=int8)

In [60]:
score(ans)

[[1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]]


0

In [10]:
def rebuildChromosome(C, hole=None):
    C = C.reshape(10,3)
    
    C[:,:-1] = C[:,:-1] % 7
    C[:, -1] = C[:, -1] % 4
    
    return C

In [11]:
def stepChromosomes(C, hole=None):
    old_C = C.flatten()
    C_list = []
    iter_size = old_C.size
    if hole:
        iter_size -=3
    for x in range(iter_size):
        new_C_add = old_C.copy()
        new_C_sub = old_C.copy()
        
        new_C_add[x] += 1
        new_C_sub[x] -= 1
        
        new_C_add = rebuildChromosome(new_C_add).copy()
        new_C_sub = rebuildChromosome(new_C_sub).copy()
        
        if new_C_add.tostring() not in history:
            addChromosomeToHistory(new_C_add)
            C_list.append(new_C_add.copy())
        
        if new_C_sub.tostring() not in history:
            addChromosomeToHistory(new_C_sub)
            C_list.append(new_C_sub.copy())
        
    return C_list

In [12]:
C_list = [buildRandomChromosome(hole=h) for _ in range(10000)]

In [13]:
best_C = null_C.copy()
best_score = score(best_C)

In [14]:
for C in C_list:
    s = score(C)
    if s < best_score:
        best_C = C.copy()
        best_score = s
print(f'Search initialized with best score of: {best_score}.',end='')

Search initialized with best score of: 32.

In [15]:
def startSearch(hole=None):
    C_list = [buildRandomChromosome(hole=hole) for _ in range(10000)]
    best_C = null_C.copy()
    best_score = score(best_C)
    
    for C in C_list:
        s = score(C)
        if s < best_score:
            best_C = C.copy()
            best_score = s
    print(f'Search initialized with score of: {best_score}.')
    return [best_C]

In [16]:
best_C_list = [best_C]

In [17]:
i = 0

In [ ]:
best_ever = best_score
best_chrom_ever = best_C.copy()
while(best_ever != 0):
    C_list = []
    for C in best_C_list:
        C_list = C_list + stepChromosomes(C, hole=h)
    two_steps = []
    for C in C_list:
        two_steps = two_steps + stepChromosomes(C, hole=h)
    C_list = C_list + two_steps
    best_C_list = []
    if len(C_list) == 0:
        print(f'Search ended with best score of {best_score}.')
        print(f'Restarting. {len(history):,} board positions searched, best score {best_ever}.  Loop: {i:,}.')
        best_C_list = startSearch(hole=h)
        best_score=np.infty
    for C in C_list:
        s = score(C)
        if s < best_score:
            best_C_list.append(C.copy())
            best_score = s
            if best_score < best_ever:
                best_ever = best_score
                best_chrom_ever = C.copy()
                print(f'New best score found with score = {best_ever}.')
    i += 1

New best score found with score = 26.
New best score found with score = 24.
New best score found with score = 22.
New best score found with score = 20.
New best score found with score = 18.
New best score found with score = 16.
New best score found with score = 14.
New best score found with score = 12.
New best score found with score = 10.
New best score found with score = 8.
Search ended with best score of 8.
Restarting. 19,875 board positions searched, best score 8.  Loop: 5.
Search initialized with score of: 26.
Search ended with best score of 8.
Restarting. 38,355 board positions searched, best score 8.  Loop: 12.
Search initialized with score of: 34.
Search ended with best score of 8.
Restarting. 55,385 board positions searched, best score 8.  Loop: 17.
Search initialized with score of: 32.
Search ended with best score of 8.
Restarting. 79,392 board positions searched, best score 8.  Loop: 26.
Search initialized with score of: 30.
Search ended with best score of 8.
Restarting. 99,

Search initialized with score of: 32.
Search ended with best score of 10.
Restarting. 940,834 board positions searched, best score 6.  Loop: 283.
Search initialized with score of: 34.
Search ended with best score of 8.
Restarting. 962,156 board positions searched, best score 6.  Loop: 290.
Search initialized with score of: 28.
Search ended with best score of 10.
Restarting. 980,582 board positions searched, best score 6.  Loop: 296.
Search initialized with score of: 32.
Search ended with best score of 12.
Restarting. 999,061 board positions searched, best score 6.  Loop: 301.
Search initialized with score of: 28.
Search ended with best score of 12.
Restarting. 1,016,198 board positions searched, best score 6.  Loop: 307.
Search initialized with score of: 36.
Search ended with best score of 8.
Restarting. 1,036,020 board positions searched, best score 6.  Loop: 313.
Search initialized with score of: 30.
Search ended with best score of 18.
Restarting. 1,050,263 board positions searched, 

Search initialized with score of: 26.
Search ended with best score of 8.
Restarting. 1,865,425 board positions searched, best score 6.  Loop: 549.
Search initialized with score of: 28.
Search ended with best score of 10.
Restarting. 1,885,247 board positions searched, best score 6.  Loop: 556.
Search initialized with score of: 28.
Search ended with best score of 14.
Restarting. 1,900,989 board positions searched, best score 6.  Loop: 560.
Search initialized with score of: 24.
Search ended with best score of 10.
Restarting. 1,913,888 board positions searched, best score 6.  Loop: 563.
Search initialized with score of: 28.
Search ended with best score of 12.
Restarting. 1,928,182 board positions searched, best score 6.  Loop: 567.
Search initialized with score of: 26.
Search ended with best score of 12.
Restarting. 1,943,769 board positions searched, best score 6.  Loop: 571.
Search initialized with score of: 24.
Search ended with best score of 12.
Restarting. 1,959,356 board positions s

Search initialized with score of: 32.
Search ended with best score of 14.
Restarting. 2,770,740 board positions searched, best score 6.  Loop: 805.
Search initialized with score of: 20.
Search ended with best score of 10.
Restarting. 2,784,983 board positions searched, best score 6.  Loop: 808.
Search initialized with score of: 26.
Search ended with best score of 8.
Restarting. 2,804,754 board positions searched, best score 6.  Loop: 815.
Search initialized with score of: 32.
Search ended with best score of 6.
Restarting. 2,827,215 board positions searched, best score 6.  Loop: 822.
Search initialized with score of: 24.
Search ended with best score of 6.
Restarting. 2,847,087 board positions searched, best score 6.  Loop: 829.
Search initialized with score of: 24.
Search ended with best score of 12.
Restarting. 2,862,776 board positions searched, best score 6.  Loop: 834.
Search initialized with score of: 28.
Search ended with best score of 12.
Restarting. 2,881,050 board positions sea

Search initialized with score of: 30.
Search ended with best score of 8.
Restarting. 3,725,559 board positions searched, best score 6.  Loop: 1,080.
Search initialized with score of: 32.
Search ended with best score of 12.
Restarting. 3,743,985 board positions searched, best score 6.  Loop: 1,085.
Search initialized with score of: 24.
Search ended with best score of 10.
Restarting. 3,759,674 board positions searched, best score 6.  Loop: 1,090.
Search initialized with score of: 32.
Search ended with best score of 12.
Restarting. 3,777,998 board positions searched, best score 6.  Loop: 1,095.
Search initialized with score of: 30.
Search ended with best score of 14.
Restarting. 3,792,292 board positions searched, best score 6.  Loop: 1,099.
Search initialized with score of: 26.
Search ended with best score of 8.
Restarting. 3,809,324 board positions searched, best score 6.  Loop: 1,104.
Search initialized with score of: 30.
Search ended with best score of 10.
Restarting. 3,829,145 board 

...

Search initialized with score of: 34.
Search ended with best score of 6.
Restarting. 13,857,929 board positions searched, best score 4.  Loop: 4,050.
Search initialized with score of: 30.
Search ended with best score of 12.
Restarting. 13,873,566 board positions searched, best score 4.  Loop: 4,054.
Search initialized with score of: 30.
Search ended with best score of 12.
Restarting. 13,891,993 board positions searched, best score 4.  Loop: 4,060.
Search initialized with score of: 30.
Search ended with best score of 10.
Restarting. 13,910,526 board positions searched, best score 4.  Loop: 4,066.
Search initialized with score of: 22.
Search ended with best score of 12.
Restarting. 13,923,425 board positions searched, best score 4.  Loop: 4,069.
Search initialized with score of: 34.
Search ended with best score of 8.
Restarting. 13,944,746 board positions searched, best score 4.  Loop: 4,077.
Search initialized with score of: 28.
Search ended with best score of 10.
Restarting. 13,961,936

Search initialized with score of: 30.
Search ended with best score of 10.
Restarting. 14,801,488 board positions searched, best score 4.  Loop: 4,329.
Search initialized with score of: 22.
Search ended with best score of 12.
Restarting. 14,817,178 board positions searched, best score 4.  Loop: 4,332.
Search initialized with score of: 32.
Search ended with best score of 10.
Restarting. 14,834,262 board positions searched, best score 4.  Loop: 4,338.
Search initialized with score of: 30.
Search ended with best score of 14.
Restarting. 14,848,556 board positions searched, best score 4.  Loop: 4,342.
Search initialized with score of: 30.
Search ended with best score of 10.
Restarting. 14,868,271 board positions searched, best score 4.  Loop: 4,346.
Search initialized with score of: 28.
Search ended with best score of 16.
Restarting. 14,882,565 board positions searched, best score 4.  Loop: 4,350.
Search initialized with score of: 24.
Search ended with best score of 10.
Restarting. 14,898,2

Search initialized with score of: 28.
Search ended with best score of 10.
Restarting. 15,707,946 board positions searched, best score 4.  Loop: 4,593.
Search initialized with score of: 26.
Search ended with best score of 10.
Restarting. 15,722,240 board positions searched, best score 4.  Loop: 4,597.
Search initialized with score of: 30.
Search ended with best score of 12.
Restarting. 15,739,372 board positions searched, best score 4.  Loop: 4,602.
Search initialized with score of: 38.
Search ended with best score of 18.
Restarting. 15,756,302 board positions searched, best score 4.  Loop: 4,606.
Search initialized with score of: 24.
Search ended with best score of 10.
Restarting. 15,771,991 board positions searched, best score 4.  Loop: 4,611.
Search initialized with score of: 26.
Search ended with best score of 8.
Restarting. 15,790,418 board positions searched, best score 4.  Loop: 4,617.
Search initialized with score of: 34.
Search ended with best score of 10.
Restarting. 15,807,50

Search initialized with score of: 26.
Search ended with best score of 10.
Restarting. 16,610,796 board positions searched, best score 4.  Loop: 4,844.
Search initialized with score of: 30.
Search ended with best score of 8.
Restarting. 16,629,382 board positions searched, best score 4.  Loop: 4,851.
Search initialized with score of: 34.
Search ended with best score of 12.
Restarting. 16,646,414 board positions searched, best score 4.  Loop: 4,856.
Search initialized with score of: 24.
Search ended with best score of 16.
Restarting. 16,659,313 board positions searched, best score 4.  Loop: 4,859.
Search initialized with score of: 28.
Search ended with best score of 16.
Restarting. 16,672,212 board positions searched, best score 4.  Loop: 4,862.
Search initialized with score of: 30.
Search ended with best score of 8.
Restarting. 16,690,793 board positions searched, best score 4.  Loop: 4,867.
Search initialized with score of: 34.
Search ended with best score of 16.
Restarting. 16,706,432

Search initialized with score of: 30.
Search ended with best score of 4.
Restarting. 17,511,166 board positions searched, best score 4.  Loop: 5,101.
Search initialized with score of: 28.
Search ended with best score of 8.
Restarting. 17,528,250 board positions searched, best score 4.  Loop: 5,107.
Search initialized with score of: 32.
Search ended with best score of 10.
Restarting. 17,546,729 board positions searched, best score 4.  Loop: 5,113.
Search initialized with score of: 28.
Search ended with best score of 14.
Restarting. 17,561,076 board positions searched, best score 4.  Loop: 5,117.
Search initialized with score of: 42.
Search ended with best score of 10.
Restarting. 17,579,556 board positions searched, best score 4.  Loop: 5,124.
Search initialized with score of: 34.
Search ended with best score of 12.
Restarting. 17,596,642 board positions searched, best score 4.  Loop: 5,130.
Search initialized with score of: 22.
Search ended with best score of 8.
Restarting. 17,615,122 